<a href="https://colab.research.google.com/github/aderdouri/EiCNAM/blob/master/Tutorials/Notebboks/mc_dupire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mathematical Description of the Financial Model

## Barrier Options under the Black-Scholes Model

A barrier option is a type of financial derivative whose payoff depends on whether the underlying asset's price reaches a specific barrier level during the option's life.


### Black-Scholes Formula for Barrier Options

The value of a barrier option can be derived by adjusting the standard Black-Scholes model with additional terms to account for the barrier conditions. The formulas for *up-and-out call* and *down-and-out put* options are given by:

$
C_{\text{up-out}} =
\begin{cases}
    0, & S \geq H \\\\
    S \Phi(d_1) - K e^{-rT} \Phi(d_2) - \left[ S \left(\frac{H}{S}\right)^{2\lambda} \Phi(x_1) - K e^{-rT} \left(\frac{H}{S}\right)^{2\lambda - 2} \Phi(x_2) \right], & S < H
\end{cases},
$

$
P_{\text{down-out}} =
\begin{cases}
    0, & S \leq H \\\\
    K e^{-rT} \Phi(-d_2) - S \Phi(-d_1) - \left[ K e^{-rT} \left(\frac{H}{S}\right)^{2\lambda - 2} \Phi(-x_2) - S \left(\frac{H}{S}\right)^{2\lambda} \Phi(-x_1) \right], & S > H
\end{cases},
$


### Parameter Definitions

$
d_1 = \frac{\ln(S / K) + (r + 0.5 \sigma^2)T}{\sigma \sqrt{T}}, \quad
d_2 = d_1 - \sigma \sqrt{T},
$

$
x_1 = \frac{\ln(S / H)}{\sigma \sqrt{T}} + \lambda \sigma \sqrt{T}, \quad
x_2 = x_1 - \sigma \sqrt{T},
$

$
\lambda = \frac{r + 0.5 \sigma^2}{\sigma^2}.
$

Here, $ S $ is the spot price, $ K $ is the strike price, $ H $ is the barrier level, $ r $ is the risk-free interest rate, $ T $ is the time to maturity, and $ \sigma $ is the local volatility. $ \Phi(\cdot) $ is the cumulative distribution function of the standard normal distribution.


## Sensitivity Analysis using Automatic Differentiation

The sensitivity of the barrier option price to the local volatility surface $ \sigma_{\text{loc}}(K, T) $ is computed using automatic differentiation. The sensitivity is defined as:

$
\frac{\partial V}{\partial \sigma_{\text{loc}}(K, T)},
$

where $ V $ is the price of the barrier option. By leveraging automatic differentiation, this derivative is computed directly in the computational graph without requiring finite difference approximations.


## Monte Carlo Simulation for Barrier Option Pricing

The price of a barrier option is computed using Monte Carlo simulation by simulating the paths of the underlying asset under stochastic dynamics. The stochastic differential equation for the asset price $ S_t $ is given by:

$
dS_t = r S_t \, dt + \sigma S_t \, dW_t,
$

where:
- $ r $ is the risk-free rate,
- $ \sigma $ is the local volatility,
- $ W_t $ is a Wiener process.

The asset price at each time step is simulated using the discretized equation:

$
S_{t+\Delta t} = S_t \exp\left((r - 0.5 \sigma^2) \Delta t + \sigma \sqrt{\Delta t} Z\right),
$

where \( Z \sim \mathcal{N}(0, 1) \) are standard normal random variables.

### Barrier Condition

For an **up-and-out call option**, the option is "knocked out" (value becomes zero) if:

$
S_t \geq H \quad \text{for any } t \in [0, T],
$

where $ H $ is the barrier level. Similarly, for a **down-and-out put option**, the option is knocked out if:

$
S_t \leq H \quad \text{for any } t \in [0, T].
$

### Payoff Calculation

The payoff at maturity is given by:
- For a call option: $\max(S_T - K, 0)$,
- For a put option: $\max(K - S_T, 0)$.

The discounted payoff is:

$
V = e^{-rT} \mathbb{E}[\text{Payoff}],
$

where $ \mathbb{E}[\text{Payoff}] $ is the average of the payoffs across all Monte Carlo paths.

### Sensitivity Analysis

The sensitivity of the option price to the local volatility $ \sigma $ is computed using finite differences:

$
\frac{\partial V}{\partial \sigma} \approx \frac{V(\sigma + \epsilon) - V(\sigma)}{\epsilon}.
$

Here, \( \epsilon \) is a small perturbation applied to the volatility surface.


In [ ]:
import torch

# Monte Carlo simulation for barrier option pricing
def barrier_option_price_mc(S, K, T, r, sigma, H, option_type="call", barrier_type="up-and-out", num_paths=10000):
    # Initialize Monte Carlo simulation parameters
    num_steps = 252  # Assume 252 trading days in a year
    dt = T / num_steps  # Time step size

    # Simulate asset paths
    paths = torch.zeros(num_paths, num_steps)
    paths[:, 0] = S
    for t in range(1, num_steps):
        z = torch.randn(num_paths)
        paths[:, t] = paths[:, t - 1] * torch.exp((r - 0.5 * sigma**2) * dt + sigma * torch.sqrt(dt) * z)

    # Barrier condition
    if barrier_type == "up-and-out":
        barrier_breached = torch.any(paths >= H, dim=1)
    elif barrier_type == "down-and-out":
        barrier_breached = torch.any(paths <= H, dim=1)
    else:
        raise ValueError("Unsupported barrier type")

    # Calculate payoff at maturity
    final_prices = paths[:, -1]
    if option_type == "call":
        payoffs = torch.maximum(final_prices - K, torch.tensor(0.0))
    elif option_type == "put":
        payoffs = torch.maximum(K - final_prices, torch.tensor(0.0))
    else:
        raise ValueError("Unsupported option type")

    # Set payoffs to zero if the barrier is breached
    payoffs[barrier_breached] = 0.0

    # Discount payoffs to present value
    option_price = torch.mean(payoffs) * torch.exp(-r * T)
    return option_price

# Function to calculate sensitivities for barrier options using Monte Carlo simulation
def calculate_barrier_sensitivities_mc(S, K, T, r, sigma_surface, H, option_type="call",
                                       barrier_type="up-and-out", num_paths=10000, epsilon=1e-4):
    sensitivity_sigma = torch.zeros_like(sigma_surface)

    for i in range(sigma_surface.shape[0]):  # Loop over strikes
        for j in range(sigma_surface.shape[1]):  # Loop over maturities
            sigma = sigma_surface[i, j]
            # Base price
            base_price = barrier_option_price_mc(S, K[i], T[j], r, sigma, H, option_type, barrier_type, num_paths)

            # Perturb volatility
            perturbed_price = barrier_option_price_mc(S, K[i], T[j], r, sigma + epsilon, H, option_type, barrier_type, num_paths)

            # Sensitivity using finite differences
            sensitivity_sigma[i, j] = (perturbed_price - base_price) / epsilon

    return sensitivity_sigma

# Example parameters
S = torch.tensor(100.0)  # Spot price
K = torch.linspace(90, 110, 10)  # Strike prices
T = torch.linspace(0.1, 2.0, 5)  # Time to maturities
r = torch.tensor(0.05)  # Risk-free rate
H = torch.tensor(120.0)  # Barrier level
sigma_surface = torch.ones((10, 5)) * 0.2  # Volatility surface

# Calculate sensitivities
sensitivity_sigma = calculate_barrier_sensitivities_mc(S, K, T, r, sigma_surface, H)

# Print the sensitivities
print("Sensitivity with respect to the volatility surface for barrier options:")
print(sensitivity_sigma)


In [ ]:
Sensitivity with respect to the volatility surface for barrier options:
tensor([[ -7.4901, -49.0454,  16.0159,  74.9784, 124.1026],
        [ -5.0400, -45.0004,  18.1770,  75.6520, 123.6662],
        [ -2.0355, -41.2216,  20.0896,  76.1119, 123.0459],
        [  0.9166, -37.8462,  21.6849,  76.3130, 122.2082],
        [  3.0453, -34.9827,  22.9082,  76.2186, 121.1256],
        [  3.7880, -32.7031,  23.7204,  75.8011, 119.7762],
        [  3.0658, -31.0389,  24.0987,  75.0421, 118.1445],
        [  1.2791, -29.9824,  24.0363,  73.9323, 116.2209],
        [ -0.9372, -29.4912,  23.5411,  72.4707, 114.0010],
        [ -3.0091, -29.4960,  22.6336,  70.6640, 111.4861]],
       grad_fn=<AddBackward0>)

In [ ]:
def barrier_option_price_mc(S, K, T, r, sigma, H, option_type="call", barrier_type="up-and-out",
                            num_paths=100000):
    num_steps = 252  # Assume 252 trading days in a year
    dt = T / num_steps  # Time step size

    # Antithetic sampling
    half_paths = num_paths // 2
    z = torch.randn(half_paths, num_steps - 1)
    z = torch.cat((z, -z), dim=0)  # Generate antithetic paths

    # Simulate asset paths
    paths = torch.zeros(num_paths, num_steps)
    paths[:, 0] = S
    for t in range(1, num_steps):
        drift = (r - 0.5 * sigma**2) * dt
        diffusion = sigma * torch.sqrt(dt) * z[:, t - 1]
        paths[:, t] = paths[:, t - 1] * torch.exp(drift + diffusion)

    # Barrier condition
    if barrier_type == "up-and-out":
        barrier_breached = torch.any(paths >= H, dim=1)
    elif barrier_type == "down-and-out":
        barrier_breached = torch.any(paths <= H, dim=1)
    else:
        raise ValueError("Unsupported barrier type")

    # Calculate payoff at maturity
    final_prices = paths[:, -1]
    if option_type == "call":
        payoffs = torch.maximum(final_prices - K, torch.tensor(0.0))
    elif option_type == "put":
        payoffs = torch.maximum(K - final_prices, torch.tensor(0.0))
    else:
        raise ValueError("Unsupported option type")

    # Set payoffs to zero if the barrier is breached
    payoffs[barrier_breached] = 0.0

    # Discount payoffs to present value
    option_price = torch.mean(payoffs) * torch.exp(-r * T)
    return option_price


In [ ]:
# Function to calculate sensitivities for barrier options using Monte Carlo simulation
def calculate_barrier_sensitivities_mc(S, K, T, r, sigma_surface, H, option_type="call", barrier_type="up-and-out", num_paths=10000, epsilon=1e-4):
    sensitivity_sigma = torch.zeros_like(sigma_surface)

    for i in range(sigma_surface.shape[0]):  # Loop over strikes
        for j in range(sigma_surface.shape[1]):  # Loop over maturities
            sigma = sigma_surface[i, j]
            # Base price
            base_price = barrier_option_price_mc(S, K[i], T[j], r, sigma, H, option_type, barrier_type, num_paths)

            # Perturb volatility
            perturbed_price = barrier_option_price_mc(S, K[i], T[j], r, sigma + epsilon, H, option_type, barrier_type, num_paths)

            # Sensitivity using finite differences
            sensitivity_sigma[i, j] = (perturbed_price - base_price) / epsilon

    return sensitivity_sigma

# Example parameters
S = torch.tensor(100.0)  # Spot price
K = torch.linspace(90, 110, 10)  # Strike prices
T = torch.linspace(0.1, 2.0, 5)  # Time to maturities
r = torch.tensor(0.05)  # Risk-free rate
H = torch.tensor(120.0)  # Barrier level
sigma_surface = torch.ones((10, 5)) * 0.2  # Volatility surface

# Calculate sensitivities
sensitivity_sigma = calculate_barrier_sensitivities_mc(S, K, T, r, sigma_surface, H)

# Print the sensitivities
print("Sensitivity with respect to the volatility surface for barrier options:")
print(sensitivity_sigma)
